In [1]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [2]:
#!git clone https://github.com/lancopku/codable-watermarking-for-llm
!pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.7 MB/s eta 0:00:00


In [3]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [4]:
cd wang-codable-watermarking-for-llm

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/wang-codable-watermarking-for-llm


In [5]:
!pip install -q -r new_requirements.txt
!pip install -q importlib datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [6]:
from importlib import reload
import torch
import pandas as pd
from tqdm import tqdm
import os
import watermarking.watermark_processor

from watermarking.watermark_processor import RepetitionPenaltyLogitsProcessor
from transformers import LogitsProcessorList, MinLengthLogitsProcessor, LogitsProcessor, AutoTokenizer, AutoModelForCausalLM
from watermarking.utils.text_tools import truncate
from watermarking.utils.load_local import load_local_model_or_tokenizer
from datasets import load_dataset, load_from_disk

import watermarking.watermark_processors.message_models.lm_message_model
import watermarking.watermark_processors.message_model_processor

from watermarking.watermark_processors.message_models.lm_message_model import LMMessageModel
from watermarking.watermark_processors.message_model_processor import WmProcessorMessageModel

In [7]:
checkpoint = 'facebook/opt-1.3b'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [8]:
min_length_processor = MinLengthLogitsProcessor(min_length=1000,
                                                eos_token_id=tokenizer.eos_token_id)
repetition_processor = RepetitionPenaltyLogitsProcessor(penalty=1.5)

watermarking.watermark_processor = reload(watermarking.watermark_processor)

watermarking.watermark_processors.message_models.lm_message_model = reload(watermarking.watermark_processors.message_models.lm_message_model)

watermarking.watermark_processors.message_model_processor = reload(watermarking.watermark_processors.message_model_processor)

In [9]:
cd '../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [10]:
lm_message_model = LMMessageModel(tokenizer=tokenizer, lm_model=model, lm_tokenizer=tokenizer,
    delta = 1.1, lm_prefix_len=10, lm_topk=-1, message_code_len = 20, random_permutation_num=50)
wm_precessor_message_model = WmProcessorMessageModel(message_model=lm_message_model, tokenizer=tokenizer,
    encode_ratio=5, max_confidence_lbd=0.5, strategy='max_confidence', message=[42,34,54,665,226,329])

In [11]:
import string
p = set(string.printable)

In [ ]:
batch_size = 4
filename = 'wang_no_attack.json'

df = pd.read_json('c4_selection.json', lines=True)

with open(filename, "r+" if os.path.exists(filename) else "w+") as f:
    with torch.no_grad():
        for i in tqdm(range(len(f.readlines()), len(df["input"]), batch_size)):
            line = df["input"][i:i+batch_size].values.tolist()
            tokenized_input = tokenizer(line, return_tensors='pt').to(model.device)

            start_length = tokenized_input['input_ids'].shape[-1]

            wm_precessor_message_model.start_length = start_length

            output_tokens = model.generate(**tokenized_input, max_new_tokens=250, num_beams=4,
                               logits_processor=LogitsProcessorList(
                                   [min_length_processor, repetition_processor,
                                    wm_precessor_message_model]))

            # if decoder only model, then we need to isolate the
            # newly generated tokens as only those are watermarked, the input/prompt is not

            for j in range(len(output_tokens)):
                output_text = tokenizer.decode(output_tokens[j][51:], skip_special_tokens=True)

                _line = ''.join(filter(lambda x: x in p, line[j])).replace('\n', '\\n').replace('"', '\\"')
                _output_text = ''.join(filter(lambda x: x in p, output_text)).replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "continuation": "{_output_text}"}}\n')

  0%|          | 0/48 [00:00<?, ?it/s]/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/wang-codable-watermarking-for-llm/watermarking/watermark_processors/message_model_processor.py:183: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:232.)
  scores.scatter_(1, topk_indices, log_Ps, reduce='add')
 73%|███████▎  | 35/48 [1:12:09<26:55, 124.25s/it]